In [ ]:
from utils.process_utils import *

In [ ]:
# 从原始的EEG信号中获取到每个session的时间，并对speech的时间进行检测冲突，获得可以使用的被试的数据。
dataset_validation = 0
dataset_validation_list = []
processor = PreprocessorWithAudio()
formal_data_root_dir = './formal_dataset'
for number in tqdm(range(1, 101)):
    formal_data_subject_root_dir = os.path.join(formal_data_root_dir, f'sub_{number}')
    eeg_path = processor.get_eeg_path(number)
    if eeg_path is None:
        dataset_validation_list.append((number, -1))
    else:
        try:

            audio_path_list = processor.get_audio_path(number)
            processor.load_data(eeg_path)
            processor.get_info(number)
            # 保存原始数据 暂时不弄这个

            orig_eeg = processor.raw.get_data()[:66]
            orig_eeg_path = os.path.join(formal_data_root_dir, f'orig_eeg/orig_eeg_sub_{number}.npy')
            os.makedirs(os.path.dirname(orig_eeg_path), exist_ok=True)
            np.save(orig_eeg_path, orig_eeg)

            processor.filter_data()
            processor.fix_bads()
            # 将处理好的数据保存起来
            pro_eeg = processor.raw.get_data()[:66]
            processed_eeg_path = os.path.join(formal_data_subject_root_dir, f'pro_eeg/pro_eeg_sub_{number}.npy')
            os.makedirs(os.path.dirname(processed_eeg_path), exist_ok=True)
            np.save(processed_eeg_path, pro_eeg)
            # 保存info
            processed_eeg_info_path = os.path.join(formal_data_subject_root_dir,
                                                   f'pro_eeg/pro_eeg_info_sub_{number}.npy')
            os.makedirs(os.path.dirname(processed_eeg_info_path), exist_ok=True)
            np.save(processed_eeg_info_path, processor.info)
            # 保存ch_names
            processed_eeg_ch_names_path = os.path.join(formal_data_subject_root_dir,
                                                       f'pro_eeg/pro_eeg_ch_names_sub_{number}.npy')
            os.makedirs(os.path.dirname(processed_eeg_ch_names_path), exist_ok=True)
            np.save(processed_eeg_ch_names_path, np.array(processor.raw.ch_names))

            # 进入分割操作
            validate = check_audio_eeg(processor.info, audio_path_list)
            dataset_validation += validate
            dataset_validation_list.append((number, validate))
            sub_video_dict = {}
            sub_speech_dict = {}
            if validate == 1:
                for seg_idx in range(8):
                    formal_data_seg_root_dir = os.path.join(formal_data_subject_root_dir, f'seg_{seg_idx}')
                    # 从脑电信号中得到分段的时间 audio
                    seg_time_idx_s, seg_time_idx_e = get_audio_segment_from_idx(processor.info, seg_idx)
                    audio_path = audio_path_list[seg_idx]
                    waveform, sample_rate = sf.read(audio_path)
                    waveform = waveform[:, 0]
                    wave_sec = waveform.shape[0] / sample_rate
                    # get the spoken eeg signal and save
                    eeg_signal = pro_eeg[:, seg_time_idx_s:seg_time_idx_s + int(wave_sec * processor.eeg_sr)]
                    processed_spoken_eeg_seg_path = os.path.join(formal_data_seg_root_dir,
                                                                 f'eeg/sub_{number}_seg_{seg_idx}_spoken_eeg.npy')
                    os.makedirs(os.path.dirname(processed_spoken_eeg_seg_path), exist_ok=True)
                    np.save(processed_spoken_eeg_seg_path, eeg_signal)

                    sub_speech_dict[seg_idx] = {
                        'duration': wave_sec,
                        'start_sec': seg_time_idx_s / processor.eeg_sr,
                        'end_sec': seg_time_idx_s / processor.eeg_sr + wave_sec,
                    }
                    # get the audio signal and save
                    processed_audio_path = os.path.join(formal_data_seg_root_dir,
                                                        f'audio/sub_{number}_seg_{seg_idx}_audio.npy')
                    os.makedirs(os.path.dirname(processed_audio_path), exist_ok=True)
                    np.save(processed_audio_path, waveform)
                    # 观看视频时的数据存储
                    seg_watching_time_idx_s, seg_watching_time_idx_e = get_segment_from_idx(processor.info, seg_idx)
                    eeg_watching_signal = pro_eeg[:, seg_watching_time_idx_s:seg_watching_time_idx_e]
                    processed_watching_eeg_seg_path = os.path.join(formal_data_seg_root_dir,
                                                                   f'eeg/sub_{number}_seg_{seg_idx}_watching_eeg.npy')
                    os.makedirs(os.path.dirname(processed_watching_eeg_seg_path), exist_ok=True)
                    np.save(processed_watching_eeg_seg_path, eeg_watching_signal)

                    # 观看的视频名字
                    video_duration, video_name = processor.get_watch_video_duration(number=number, seg_idx=seg_idx,
                                                                                    return_video_name=True)
                    sub_video_dict[seg_idx] = {
                        'name': video_name,
                        'duration': video_duration,
                        'start_sec': seg_watching_time_idx_s / processor.eeg_sr,
                        'end_sec': seg_watching_time_idx_e / processor.eeg_sr,
                    }
                # this is wrong, fix this and delete this.
                # save meta data
                meta_dict = {
                    'ch_name': processor.raw.ch_names,
                    'info': processor.info[:, 0].tolist(),
                    'spoken_eeg_path': processed_spoken_eeg_seg_path,
                    'audio_path': processed_audio_path,
                    'watching_eeg_seg_path': processed_watching_eeg_seg_path,
                    'video_name': sub_video_dict,
                    'eeg_sr': processor.eeg_sr,
                    'speech_sr': processor.audio_sr,
                }
                meta_output_path = os.path.join(formal_data_subject_root_dir, f'meta_info.json')
                with open(meta_output_path, "w") as json_file:
                    json.dump(meta_dict, json_file, indent=4, ensure_ascii=False)
        except Exception as e:
            print(number, e)
            traceback.print_exc()
            print('\n' * 2)
            dataset_validation_list.append((number, 2))


In [ ]:
# 检查哪些被试的数据是可用的
    
valid_subject_list=[]
for i,subject_number in tqdm(enumerate(range(1,101))):
    seg_idx=0
    try:
        npy_path=f'formal_dataset/sub_{subject_number}/meta_info.json'
        with open(npy_path,'r',encoding='utf8')as fp:
            json_data = json.load(fp)
        # print(subject_number)
    except:
        continue
    valid_subject_list.append(subject_number)    
print(len(valid_subject_list))

In [ ]:
meta_output_path='formal_dataset/valid.json'
meta_dict={
    'valid_number':valid_subject_list,
    'total_valid_number':len(valid_subject_list),
}
with open(meta_output_path, "w") as json_file:
    json.dump(meta_dict, json_file, indent=4,ensure_ascii=False)

In [ ]:
meta_output_path='formal_dataset/valid.json'
with open(meta_output_path, "r") as json_file:
    meta_dict=json.load(json_file)

In [ ]:
model=whisper.load_model('large',download_root='./whisper_models')

In [ ]:
valid_subject_list=meta_dict['valid_number']

In [ ]:
# 从音频转写到文本。还需要把音频转为16kHz，写入json文件
for subject_number in tqdm(valid_subject_list):
    for seg_idx in tqdm(range(8)):
        npy_path=f'formal_dataset/sub_{subject_number}/seg_{seg_idx}/audio/sub_{subject_number}_seg_{seg_idx}_audio.npy'
        signal=np.load(npy_path)
        sample_rate=44100
        new_sample_rate=16000
        new_signal = librosa.resample(signal, orig_sr=sample_rate, target_sr=new_sample_rate)
        # try:
        #     os.remove(npy_path[:-4]+'.mp3')
        #     print("文件已成功删除！")
        # except OSError as e:
        #     print("删除文件时出现错误:", e)
        processed_audio_path=npy_path[:-4]+'16kHz.mp3'
        sf.write(processed_audio_path, new_signal, new_sample_rate)
        result = model.transcribe(
            processed_audio_path,language="Chinese",word_timestamps=True,
            without_timestamps=False,initial_prompt='以下是简体中文的句子，每个句子只能有一个标点符号。')

        # 写入文本
        transcribe_path=f"./transcribe/sub_{subject_number}/seg_{seg_idx}/transcribe_sub_{subject_number}_seg_{seg_idx}.json"
        transcribe_path=makedirs(transcribe_path)
        with open(transcribe_path, 'w') as write_f:
        	json.dump(result, write_f, indent=4, ensure_ascii=False)
        with open(npy_path[:-3]+'json', 'w') as write_f:
        	json.dump(result, write_f, indent=4, ensure_ascii=False)
        

In [ ]:
# 统计有多少个词汇
# calculate total vocabulary
word_dict={}
char_dict={}
for subject_number in tqdm(valid_subject_list):
    for seg_idx in range(8):
        transcribe_path=f"./transcribe/sub_{subject_number}/seg_{seg_idx}/transcribe_sub_{subject_number}_seg_{seg_idx}.json"
        with open(transcribe_path, 'r') as file:
            data = json.load(file)
        text=data['text']
        sententce_word_dict=stat_word_freq(text)
        sententce_char_dict=stat_char_freq(text)
        word_dict=merge_dicts(word_dict,sententce_word_dict)
        char_dict=merge_dicts(char_dict,sententce_char_dict)
print(len(word_dict),len(char_dict))        
char_num=0
for key in char_dict.keys():
    char_num+=char_dict[key]
print(char_num)

In [ ]:
# 统计时长，语句数量，词汇数量，字数量。训练集和测试集的重叠数量。

In [ ]:
# 对已经正确的被试，只用word，先全部转为简体，根据标点符号分割句子，或者word太长重新成句。分割成30s以内的句子。
time_limit=5
speech_sr=16000
eeg_sr=1000
data_dict_list=[]
output_root_dir='formal_dataset'
output_folder=f'cut_seg{time_limit}s_singe_sentence'
problematic_datas=[]
try:
    # 清空之前的文件
    os.system(f'rm -rf {output_root_dir}/{output_folder}')
except Exception as e:
    pass
for subject_number in tqdm(valid_subject_list):
    data_dict_subject_list=[]
    for seg_idx in range(8):
        data_dict_seg_list=[]
        transcribe_path=f"./transcribe/sub_{subject_number}/seg_{seg_idx}/transcribe_sub_{subject_number}_seg_{seg_idx}.json"
        with open(transcribe_path, 'r') as file:
            data = json.load(file)
        text=data['text']
        segments=data['segments']
        # print(f'text:{text}')
        # if has_multiple_english_words(text):
        #     print(f'sub_{subject_number}_seg_{seg_idx} has_multiple_english_words')
        #     print(f'text:{text}')
        #     print('='*40)
        # if check_repeat_sentences(segments):    
        #     # 如果有重复的句子，我就直接删掉之前的文件，并且跳过。
        #     print(f'sub_{subject_number}_seg_{seg_idx}text:{text} has repeat sentences')
        #     continue
        status_has_multiple_english_words=has_multiple_english_words(text)
        status_has_repeat_sentences=check_repeat_sentences(segments)
        if status_has_multiple_english_words or status_has_repeat_sentences:
            problematic_datas.append([
                subject_number,
                seg_idx,
                text,
                (status_has_multiple_english_words,status_has_repeat_sentences)
            ])
            continue
            
        words=[]
        for sentence in segments:
            word=sentence['words']
            if len(word)>=1:
                word[-1]['word']=check_chinese_punctuation(word[-1]['word'],chinese_punctuation = '[,.?，。？]')
            # 如果最后一个词没有标点符号，就加句号。
            words.extend(word)
        # 从单词分成句子。
        new_sentences=combine_words_to_sentences_within_time_limit(words,time_limit=time_limit-0.1)
        # print('after combine_sentences')
        # print(new_sentences)

        # 根据时间限制分段
        # new_sentences_combination=combine_sentences_within_time_limit(new_sentences,time_limit)
        new_sentences_combination=combine_single_sentences(new_sentences)
        # print('after combine_sentences_within_time_limit')
        # print(new_sentences_combination)
        new_sentences_combination=combinations_to_data_dict(new_sentences_combination)
        # print('after combinations_to_data_dict')
        # print(new_sentences_combination)
        # 写入音频和EEG。并且写jsonlines

        # speech和EEG存放位置
        orig_eeg_path=f'{output_root_dir}/sub_{subject_number}/seg_{seg_idx}/eeg/sub_{subject_number}_seg_{seg_idx}_spoken_eeg.npy'
        orig_speech_path=f'{output_root_dir}/sub_{subject_number}/seg_{seg_idx}/audio/sub_{subject_number}_seg_{seg_idx}_audio16kHz.mp3'
             
        waveform, sample_rate = sf.read(orig_speech_path)
        assert len(waveform.shape)==1
        eeg=np.load(orig_eeg_path)
        assert eeg.shape[0]<eeg.shape[1], '通道数肯定会小于EEG文件的长度'
        
        for comb_idx,comb in enumerate(new_sentences_combination):
            # 控制起止时间和位置
            start=comb['start']
            end=comb['end']
            if end-start>29 or end-start<0.2:
                print(f'subject{subject_number} seg_idx{seg_idx} comb_idx{comb_idx} is too long'
                       f'duration is {np.round(end-start,2)}s, comb is {comb}')
            start_speech=int(start*speech_sr)
            start_eeg=int(start*eeg_sr)
            end_speech=int(end*speech_sr)
            end_eeg=int(end*eeg_sr)
            
            # 控制路径
            new_eeg_path=f'{output_root_dir}/{output_folder}/sub_{subject_number}/seg_{seg_idx}/eeg/sub_{subject_number}_seg_{seg_idx}_cut_{comb_idx}_spoken_eeg.npy'
            makedirs(new_eeg_path)
            new_speech_path=f'{output_root_dir}/{output_folder}/sub_{subject_number}/seg_{seg_idx}/speech/sub_{subject_number}_seg_{seg_idx}_cut_{comb_idx}_spoken_speech.mp3'
            makedirs(new_speech_path)
            # 切分音频和EEG
            cut_audio=waveform[start_speech:end_speech]
            cut_eeg=eeg[:,start_eeg:end_eeg]
            # 写入音频和EEG
            sf.write(new_speech_path,cut_audio,speech_sr)
            np.save(new_eeg_path,cut_eeg)
            
            # 生成json字典，安排地址，重新计算时间
            new_start=0
            new_end=end-start
            data_dict={
               "speech": {
                  "path": new_speech_path
               },
               "eeg": {
                  "path": new_eeg_path
               },
               "sentence": comb['sentence'],
               "language": "Chinese",
               "sentences": correct_timing(comb['sentences']),
               "duration": comb['duration']
            }
            data_dict_seg_list.append(data_dict)
            data_dict_subject_list.append(data_dict)
            data_dict_list.append(data_dict)
        if len(data_dict_seg_list)>0:
            write_jsonlines(f'{output_root_dir}/{output_folder}/sub_{subject_number}/seg_{seg_idx}/sub_{subject_number}_seg_{seg_idx}.jsonl',data_dict_seg_list)
        else:
            problematic_datas.append([
                subject_number,
                seg_idx,
                text,
                (2,2)
            ])
        
# print(data_dict)            
write_jsonlines(f'{output_root_dir}/{output_folder}/all_data.jsonl', data_dict_list)
# 数据处理完毕

In [ ]:

write_jsonlines(f'{output_root_dir}/{output_folder}/all_wrong_data.jsonl', problematic_datas)
# 43个有错的seg，先不管了，数据够了。
# 保存一下错误数据，之后方便修正数据。

In [ ]:
# 对数据进行打乱并切分

import numpy as np
json_dicts=read_jsonlines(f'{output_root_dir}/{output_folder}/all_data.jsonl')
np.random.shuffle(json_dicts)
all_num=len(json_dicts)
train_num=int(0.8*all_num)
val_num=int(0.9*all_num)
train_json_dicts=json_dicts[:train_num]
val_json_dicts=json_dicts[train_num:val_num]
test_json_dicts=json_dicts[val_num:]
write_jsonlines(f"{output_root_dir}/{output_folder}/train_data.jsonl",train_json_dicts)
write_jsonlines(f"{output_root_dir}/{output_folder}val_data.jsonl",val_json_dicts)
write_jsonlines(f"{output_root_dir}/{output_folder}/test_data.jsonl",test_json_dicts)

In [ ]:
all_duration=0
for i in range(len(data_dict_list)):
    all_duration=all_duration+data_dict_list[i]['duration']
print(all_duration/3600)
# 粗略的处理之后还剩 12.283661111111154个小时。

In [ ]:
# 提取一个人的数据
subject_id=1
sub_jsonl=[]
for i in range(8):
    jsonl_path=f"/home/yyang/dataset/multi_media/formal_dataset/cut_seg10s_singe_sentence/sub_1/seg_{i}/sub_1_seg_{i}.jsonl"
    try:
        jsonl_data=read_jsonlines(jsonl_path)
        sub_jsonl.extend(jsonl_data)
        print(f'successfully add seg {i}')
    except FileNotFoundError as e:
        raise e 
print(len(sub_jsonl))